In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.schema import Document
import gradio as gr

In [2]:
with open("/Users/sanjayashastry/Documents/Sanjaya/coding/Chatbot files/AM_scraped.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [4]:
print(text)


--- PAGE: https://avanimudradesign.com/ ---
Architects in Bangalore | Avani Mudra Design Studio
Skip to content
ABOUT
WORK
CONTACT US
Previous
Viraam House
Avani Mudra Design Office
Eya Aesthetic Clinic
Geleyara Goodu
Viraam House
Avani Mudra Design Office
Eya Aesthetic Clinic
Geleyara Goodu
Viraam House
Next
1
2
3
4
All Rights Reserved
Instagram
LinkedIn


--- PAGE: https://avanimudradesign.com/projects/eya-aesthetic-clinic/ ---
Eya Aesthetic Clinic – Architects in Bangalore | Avani Mudra Design Studio
ABOUT
WORK
CONTACT US
Eya Aesthetic Clinic
Client
Dr. Ritu & Ravi Ganjigatti
Location
Nagarabavi,  Bengaluru
Site Area
600 SQFT
TBUA
1,800 SQFT
Photography
Studio Recall
The site was a narrow corner plot, deep inset in the locality of Nagarabavi, dotted by huge trees. A trapezoidal plot measuring 6’ on one side and 22 ‘ on the other side with an elaborate brief to create a modern looking dental prosthetic clinic that needs to become a landmark by its architectural expression  was a rea

In [5]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = [Document(page_content=text)]
chunks = splitter.split_documents(docs)

In [6]:
chunks = [doc for doc in chunks if doc.page_content and doc.page_content.strip()]

In [7]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/var/folders/s7/l_67srrj5wq6vb6q61pc39vm0000gn/T/ipykernel_1939/323160131.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [8]:
vectorstore = Chroma.from_documents(documents=chunks, embedding=embedding, persist_directory="./am_chatbot_db")

/opt/anaconda3/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [9]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [10]:
llm = ChatOllama(model="llama3.2:latest")

/var/folders/s7/l_67srrj5wq6vb6q61pc39vm0000gn/T/ipykernel_1939/1403308041.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3.2:latest")


In [11]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

/var/folders/s7/l_67srrj5wq6vb6q61pc39vm0000gn/T/ipykernel_1939/3103747979.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)


In [12]:
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm,retriever=retriever,memory=memory)

In [13]:
def chat(message, history):
    result = conversation_chain.invoke({"question": message})
    return result["answer"]

In [ ]:
gr.ChatInterface(fn=chat, title="AM Chatbot (Local Ollama)", type="messages").launch(share=True)

* Running on local URL:  http://127.0.0.1:7860


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
